<a href="https://colab.research.google.com/github/Ravisxcr/500-AI-Machine-learning-Deep-learning-Computer-vision-NLP-Projects-with-code/blob/main/tomatao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! ls

In [ ]:
!cd PlantVillage/

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d arjuntejaswi/plant-village

In [ ]:
!unzip /content/plant-village.zip

In [ ]:
!mkdir /content/PlantVillage/Tomato

In [ ]:
! mv /content/PlantVillage/Tomato_Bacterial_spot /content/PlantVillage/Tomato

In [ ]:
!mv /content/PlantVillage/Tomato_Early_blight /content/PlantVillage/Tomato

In [ ]:
! mv /content/PlantVillage/Tomato_Late_blight /content/PlantVillage/Tomato

In [ ]:
! mv /content/PlantVillage/Tomato_Leaf_Mold /content/PlantVillage/Tomato

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [ ]:
b_size  = 36
img_size = 256
channel = 3
epoch = 20

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/PlantVillage/Tomato',
    seed=42,
    batch_size=b_size,
    image_size=(img_size,img_size),
    shuffle=True
)

In [ ]:
dataset.class_names[1]

In [ ]:
len(dataset)

In [ ]:
plt.figure(figsize=(12,10))
for image_batch, label_batch in dataset.take(1):
    for i in range(12):
        ax = plt.subplot(3,4,i+1)
        plt.imshow(image_batch[i].numpy().astype('uint8'))
        plt.title(dataset.class_names[label_batch[i]])
        plt.axis('off')


In [ ]:
def data_partition(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split +val_split + test_split) == 1

    ds_size = len(ds)

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(train_split*ds_size)
    test_size = int(test_split*ds_size)

    train_ds = ds.take(train_size)
    test_ds = ds.skip(train_size).take(test_size)
    val_ds = ds.skip(train_size).skip(test_size)

    return train_ds, test_ds, val_ds

In [ ]:
train_ds, test_ds, val_ds = data_partition(dataset)

print(len(train_ds))
print(len(test_ds))
print(len(test_ds))

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resize_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(img_size,img_size),
    layers.experimental.preprocessing.Rescaling(1./255)
])

In [ ]:
data_augmantation  = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
input_shape = (b_size, img_size, img_size, channel)
n_classes = 4

model = models.Sequential([
    resize_rescale,
    layers.Conv2D(32, kernel_size= (3,3), activation='relu', input_shape= input_shape),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64,  kernel_size= (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    batch_size=b_size,
    validation_data= val_ds,
    verbose=1,
    epochs=epoch
)

In [ ]:
model.save('/content/drive/MyDrive/colab_notebook/Deep_learning/models/tomato_model')

In [ ]:
history.params

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(epoch), acc, label='Training Accuracy')
plt.plot(range(epoch), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(epoch), loss, label='Training Loss')
plt.plot(range(epoch), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/colab_notebook/Deep_learning/models/tomato_model')

In [ ]:
score = model.evaluate(test_ds)
score

In [ ]:
import numpy as np

def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i])
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = dataset.class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_ds:
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow((images[i]/255.0))
        
        predicted_class, confidence = predict(model, images[i])
        actual_class = dataset.class_names[int(labels[i])] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")
    break